In [1]:
import os
import pandas as pd
from numpy.random import random
from chatbot import QWENChatBot
from pandarallel import pandarallel
import time
import random
from concurrent.futures import ThreadPoolExecutor

In [2]:
data = pd.read_excel('processed_data/raw_data_filtered.xlsx')
data.head()

,id,数据类型,分析对象ID,分析对象,站点id,站点名称,标题,内容,视频封面内容识别,音频内容识别,...,视频时长,图片上的标签列表,笔记里的标签列表,笔记中艾特的用户列表,合作品牌,用户地址,认证简介,MCN机构名称,用户所获点赞和收藏数,classification
0,679260ba000000001800b050,电商笔记,58153241,分析对象1,146510,小红书,🇬🇧Recently plog🍛🥞💆🏻‍,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pan...,②小红书 HEYTEA|③POPHUART 延迟过万圣 POPMART Halloeen...,NaN,...,NaN,NaN,留学英国|发点生活边角料|清清日常库存|留学plog|伦敦brunch|labubu,NaN,NaN,广东广州,NaN,NaN,21000.0,0
1,6791f0c50000000029029680,电商笔记,58153241,分析对象1,146510,小红书,阿米的新年六位数购物开箱来了🎁,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下...,①UITTON LcUSvUmo 小红书 UITTON |②DUISVUITTON LO...,NaN,...,NaN,NaN,购物开箱分享|LV|lv村上隆|miumiu|chanel|新年礼物|乘光lumihood|...,NaN,NaN,中国,NaN,NaN,58000.0,0
2,6779f53f000000000b017934,电商笔记,58153241,分析对象1,146510,小红书,lv 村上隆限时店～还有2天！,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）...,① LOUISVUITTON MURAKAMI|② ...,NaN,...,NaN,NaN,时尚潮人打卡地|随处走走就是潮流|潮流新地标,NaN,NaN,海外美国,NaN,NaN,71045.0,0
3,6784e9b10000000019034092,电商笔记,58153241,分析对象1,146510,小红书,终于到手了❗黑三彩carryall黑武士不要太好,开箱めLv村上隆包包■黑三彩Carryall 联名款黑武士羊皮carryall开箱分享 真的...,① 小红书 LOUISVUITON PARS,NaN,...,54.0,NaN,lv村上隆|LV黑三彩|lvcarryall|carryal新款|carryall|lv新款...,NaN,NaN,NaN,NaN,NaN,NaN,0
4,677953cd0000000013018d00,电商笔记,58153241,分析对象1,146510,小红书,Lv村上隆限定✖️白三彩卡包🆚黑三彩钱包,好可爱呀！兼并了美貌和实用#卡包推荐[话题]##钱包推荐[话题]# #lv[话题]# #l...,NaN,NaN,...,16.0,NaN,卡包推荐|钱包推荐|lv|lv村上隆联名|村上隆|lv白三彩|lv黑三彩,NaN,NaN,日本 东京,NaN,NaN,600.0,0


In [3]:
def qwen_tagging(comment):
    chatbot = QWENChatBot('你是一个专业的数据标注助手，专注于从社交媒体用户生成内容（UGC）中提取和标注特定的信息。你需要针对小红书上关于LVMH和村上隆合作新系列的反馈分析项目提取结构化信息')
    prompt = """

任务：
请根据以下用户生成内容（UGC）评论，按照本项目预定义的标签标准，提取与LVMH × 村上隆合作新品系列内容关联的结构化信息。所有输出必须符合 JSON 格式规范，确保输出结构完全一致.

1.确保输出内容清晰明确，不能出现模糊或不确定的情况。
2.JSON 结构中的每个字段都必须严格按照给定选项填写。如果某个选项没有涉及到，请使用空字符串 "" 或空列表 [] 表示。
3.对于有预定义分类的字段，请根据以下规则填写：
年龄范围（Age_Range）: 只能填写 "18岁以下"、"18-24岁"、"25-34岁"、"35-44岁"、"45岁以上" 或 ""（当无法判断时）。
性别（Gender）: 只能填写 "男性"、"女性" 或 ""（当无法判断时）。
城市等级（City_Tier）: 只能填写 "一线"、"二线"、"三线及以下" 或 ""（当无法判断时）。
4.对于 购买需求（Purchase_Purpose） 和 购买情境（Purchase_Situation）：
购买需求（Purchase_Purpose）:根据帖子内容，对用户为什么要购买/购买产品的动机进行分类，或 ""（当无法判断时）。
购买情景（Purchase_Situation）：根据帖子内容，对购买的场景进行分类，或 ""（当无法判断时）。
若评论未体现相关信息，则填写 ""。
5.对于提到的产品（Product_Mention）:
提取用户提到的LV和村上隆合作新系列的产品名称，或 []（当无法判断时）。
6.对于提到的产品（Competitor）:
提取用户提到的LV以外的竞争品牌名称，或 []（当无法判断时）。
7.对于用户的情感（Emotion）：
根据帖子内容和预期判断用户对lv联名系列的情感，或""当无法判断时）。
8.关键词提取（Keywords）: 对评论内容中提到的 设计元素（Design_Elements）、设计风格（Design_Style） 和 功能性（Functionality） 关键词分别分类。
设计元素（Design_Elements）： 包括颜色（如"红色"）、尺码（如"大号"）、以及新系列的饰品细节（如"樱花吊坠"）等。
设计风格（Design_Style）： 包括了总体风格词描述，比如"复古"、"青春"、"可爱"等。
功能性（Functionality）： 聚焦用户对产品功能性的具体评价以及期待，例如"有内袋"、"自重轻"、"百搭"等。
其他关键词（Other）： 包括无法归类至其他子类别但与新品系列相关的重要关键词列表。

输出 JSON 结构：
{
    "Age_Range": str, 
    "Gender": str,  
    "City_Tier": str,  
    "Purchase_Purpose": str,  
    "Purchase_Situation": str,  
    "Product_Mention":List[str],
    "Competitor":List[str],
    "Emotion":str,
    "Keywords": {
        "Design_Elements": List[str],  
        "Design_Style": List[str],  
        "Functionality": List[str], 
        "Other": List[str]  
    }
}


示例输入：
用户生成内容评论：

“超级喜欢这次村上隆联名系列的樱花吊坠设计，真是绝美！特别是粉色的颜色超少女心，大小也很适合日常出街。像我这种时尚控完全爱了！而且还可以斜挎，直接入手了限量款！”

示例输出：

{
    "Age_Range": "25-34岁",
    "Gender": "女性",
    "City_Tier": "一线",
    "Purchase_Purpose": "追求独特设计",
    "Purchase_Situation": "日常出街",
    "Product_Mention": ["樱花吊坠", "限量款"],
    "Competitor": [],
    "Emotion": "喜欢",
    "Keywords": {
        "Design_Elements": ["樱花吊坠", "粉色", "大小适合"],
        "Design_Style": ["少女心", "时尚"],
        "Functionality": ["斜挎"],
        "Other": ["村上隆", "联名系列"]
    }
}

你只能按照格式返回且只返回 JSON，如果你无法提取信息，按照规则返回所要求格式的 JSON即可,输出中不能出现多余的解释性文字、描述性文本或格式修饰符，例如：“根据提供的评论内容：”或“```json”。

请按照以下评论内容生成 JSON 标注：

评论内容：""" + f"{comment}"
    try:
        # Call the chatbot API and add random delay for throttling
        response = chatbot.ask(prompt)
        time.sleep(random.uniform(0.2, 0.8))  # Avoid rate limits
        return response
    except Exception as e:
        print(f"Error processing comment: {comment}\nException: {e}")
        return "{}"  # Return empty JSON on failure

def apply_parallel(column_data, max_workers=5):
    """
    Parallelizes processing over a single column using ThreadPoolExecutor.
    """
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        print("Starting parallel processing...")
        # Apply the function to each comment in parallel
        results = list(executor.map(qwen_tagging, column_data))
    return results
if 'raw_tagging' not in data.columns:
    data['raw_tagging'] = '0'
data["raw_tagging"] = apply_parallel(data["内容"].tolist(), max_workers=16)
data[['内容','raw_tagging']]


Starting parallel processing...


,内容,raw_tagging
0,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pan...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
1,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下...,"{\n ""Age_Range"": """",\n ""Gender"": ""女性"",\n..."
2,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
3,开箱めLv村上隆包包■黑三彩Carryall 联名款黑武士羊皮carryall开箱分享 真的...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
4,好可爱呀！兼并了美貌和实用#卡包推荐[话题]##钱包推荐[话题]# #lv[话题]# #l...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
...,...,...
2622,800年没买trainer的我陪母上大人逛街顺利蹭到一双[派对R][吐舌头H][耶R] #鞋...,"{\n ""Age_Range"": """",\n ""Gender"": ""女性"",\n..."
2623,村上隆太阳花熊猫。还有比这个组合更合适的包给我们花花吗[派对R][派对R][派对R]#LV ...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
2624,#lv村上隆快闪店 太出片啦！！！今天下午去的，除了咖啡店拍的时间久点外，其他都很快，地方也...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
2625,lv村上隆联名这次出的speedy系列，比speedy30小的只有speedynano，有看...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."


In [ ]:
if 'raw_tagging' not in data.columns:
    data['raw_tagging'] = '0'
data['raw_tagging'] = data.apply(lambda x: qwen_tagging(x), axis=1)
#data.to_excel(f'../data/processed_data/raw_tagged_data.xlsx', index=False)
data

In [4]:
data[['内容','raw_tagging']].to_excel('processed_data/tagged_raw.xlsx', index=False)

In [2]:
data_tagged = pd.read_excel('processed_data/tagged_raw.xlsx')
data_tagged

,内容,raw_tagging
0,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pan...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
1,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下...,"{\n ""Age_Range"": """",\n ""Gender"": ""女性"",\n..."
2,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
3,开箱めLv村上隆包包■黑三彩Carryall 联名款黑武士羊皮carryall开箱分享 真的...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
4,好可爱呀！兼并了美貌和实用#卡包推荐[话题]##钱包推荐[话题]# #lv[话题]# #l...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
...,...,...
2622,800年没买trainer的我陪母上大人逛街顺利蹭到一双[派对R][吐舌头H][耶R] #鞋...,"{\n ""Age_Range"": """",\n ""Gender"": ""女性"",\n..."
2623,村上隆太阳花熊猫。还有比这个组合更合适的包给我们花花吗[派对R][派对R][派对R]#LV ...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
2624,#lv村上隆快闪店 太出片啦！！！今天下午去的，除了咖啡店拍的时间久点外，其他都很快，地方也...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
2625,lv村上隆联名这次出的speedy系列，比speedy30小的只有speedynano，有看...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."


In [3]:
import json
import numpy as np
def safe_json_parser(json_string):
    try:
        # Try parsing the JSON
        return json.loads(json_string)
    except json.JSONDecodeError:
        # If an error occurs, skip/return None
        return None

# Apply the function to the column of JSON data
data_tagged['parsed_json'] = data_tagged['raw_tagging'].apply(safe_json_parser)
# Optionally, drop rows where parsing failed (parsed_json is None)
data_tagged = data_tagged.dropna(subset=['parsed_json']).reset_index(drop=True)

In [4]:
#data_tagged['raw_tagging'] = data['raw_tagging'].apply(json.loads)
data_tagged['Age_Range'] = data_tagged['parsed_json'].apply(lambda x: x.get('Age_Range', np.nan) if x.get('Age_Range', '') != '' else np.nan)
data_tagged['Gender'] = data_tagged['parsed_json'].apply(lambda x: x.get('Gender', np.nan) if x.get('Gender', '') != '' else np.nan)
data_tagged['City_Tier'] = data_tagged['parsed_json'].apply(lambda x: x.get('City_Tier', np.nan) if x.get('City_Tier', '') != '' else np.nan)
data_tagged['Purchase_Purpose'] = data_tagged['parsed_json'].apply(lambda x: x.get('Purchase_Purpose', np.nan) if x.get('Purchase_Purpose', '') != '' else np.nan)
data_tagged['Purchase_Situation'] = data_tagged['parsed_json'].apply(lambda x: x.get('Purchase_Situation', np.nan) if x.get('Purchase_Situation', '') != '' else np.nan)
data_tagged['Product_Mention'] = data_tagged['parsed_json'].apply(lambda x: x.get('Product_Mention', np.nan) if x.get('Product_Mention', []) != [] else np.nan)
data_tagged['Competitor'] = data_tagged['parsed_json'].apply(lambda x: x.get('Competitor', np.nan) if x.get('Competitor', []) != [] else np.nan)
data_tagged['Keywords_Design_Elements'] = data_tagged['parsed_json'].apply(lambda x: x.get('Keywords', {}).get('Design_Elements', np.nan) if x.get('Keywords', {}).get('Design_Elements', []) != [] else np.nan)
data_tagged['Keywords_Design_Style'] = data_tagged['parsed_json'].apply(lambda x: x.get('Keywords', {}).get('Design_Style', np.nan) if x.get('Keywords', {}).get('Design_Style', []) != [] else np.nan)
data_tagged['Keywords_Functionality'] = data_tagged['parsed_json'].apply(lambda x: x.get('Keywords', {}).get('Functionality', np.nan) if x.get('Keywords', {}).get('Functionality', []) != [] else np.nan)
data_tagged['Keywords_Other'] = data_tagged['parsed_json'].apply(lambda x: x.get('Keywords', {}).get('Other', np.nan) if x.get('Keywords', {}).get('Other', []) != [] else np.nan)
data_tagged['Emotion'] = data_tagged['parsed_json'].apply(lambda x: x.get('Emotion', np.nan) if x.get('Emotion', '') != '' else np.nan)

data_tagged


,内容,raw_tagging,parsed_json,Age_Range,Gender,City_Tier,Purchase_Purpose,Purchase_Situation,Product_Mention,Competitor,Keywords_Design_Elements,Keywords_Design_Style,Keywords_Functionality,Keywords_Other,Emotion
0,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pan...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,一线,兴趣爱好,参观快闪店,NaN,NaN,NaN,NaN,NaN,"[联名快闪店, 一楼卖东西, 二楼是咖啡店, 很多人]",中立
1,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下...,"{\n ""Age_Range"": """",\n ""Gender"": ""女性"",\n...","{'Age_Range': '', 'Gender': '女性', 'City_Tier':...",NaN,女性,一线,个人喜好,购物体验,[水貂帽衫背心],"[miumiu, Chanel, 贝熙迦儿]",NaN,NaN,"[客户体验超有含金量, 购物无压力]","[村上隆联名系列, LV北京skp店, LV lounge]",喜欢
2,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[可爱],NaN,"[pop up, 快闪, 时尚潮人打卡地, 潮流新地标]",中立
3,开箱めLv村上隆包包■黑三彩Carryall 联名款黑武士羊皮carryall开箱分享 真的...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,实用,单肩斜挎,"[Carryall, 黑三彩Carryall, 黑武士羊皮carryall, 黑三彩]",NaN,"[黑三彩, 羊皮]",[黑武士],"[不压身高, 不显大, 实用, 单肩斜挎]","[村上隆, 联名款, 开箱分享]",喜欢
4,好可爱呀！兼并了美貌和实用#卡包推荐[话题]##钱包推荐[话题]# #lv[话题]# #l...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,追求美观与实用,NaN,"[卡包, 钱包, 白三彩, 黑三彩]",NaN,[美貌],[可爱],[实用],"[村上隆, 联名系列]",喜欢
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2621,800年没买trainer的我陪母上大人逛街顺利蹭到一双[派对R][吐舌头H][耶R] #鞋...,"{\n ""Age_Range"": """",\n ""Gender"": ""女性"",\n...","{'Age_Range': '', 'Gender': '女性', 'City_Tier':...",NaN,女性,NaN,陪家人购物,逛街,[trainer],NaN,NaN,NaN,NaN,"[lv村上隆, 心动的鞋款]",心动
2622,村上隆太阳花熊猫。还有比这个组合更合适的包给我们花花吗[派对R][派对R][派对R]#LV ...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,追求独特设计,NaN,"[村上隆手袋, 大熊猫花花]",NaN,[太阳花熊猫],[可爱],NaN,"[村上隆, 联名系列, LV新款]",喜欢
2623,#lv村上隆快闪店 太出片啦！！！今天下午去的，除了咖啡店拍的时间久点外，其他都很快，地方也...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,NaN,拍照,NaN,NaN,NaN,NaN,NaN,"[快闪店, 咖啡店, 出片]",喜欢
2624,lv村上隆联名这次出的speedy系列，比speedy30小的只有speedynano，有看...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,NaN,NaN,"[speedy系列, speedynano, speedy30, 黑三彩]",NaN,"[speedynano, oversiza, 上身偏大]",[酷],NaN,"[lv村上隆, 联名, lv新品]",NaN


In [12]:
data_tagged.Purchase_Purpose.unique()

array(['兴趣爱好', '个人喜好', nan, '实用', '追求美观与实用', '追求独特设计', '奖励自己', '收藏与回忆',
       '追求可爱设计', '追求特定产品', '追求品牌', '追求时尚', '被可爱到', '时尚追求', '收藏/改造', '收藏',
       '寻找合适自己的包款', '搭配需求', '日常使用', '怀旧情怀', '追求产品质量', '追求设计感', '购买限量版产品',
       '情怀', '追求舒适性和审美偏好', '收藏/预定', '追求时尚与美观', '追求热门产品', '满足个人喜好', '收藏情怀',
       '追求时尚潮流', '尝试新品', '季节性穿着', '外观吸引', '追求品质与时尚', '日常搭配', '作为礼物',
       '旅行需求', '为自己的快乐买单', '支持艺术家合作款', '实际可用性考虑', '追求经典设计', '选择困难',
       '收藏/限量版', '追求功能性', '送礼', '怀旧情感', '实用需求', '纪念过去的回忆', '追求简洁设计',
       '购买特定产品', '喜欢', '追求热门款', '追求优质材料', '未知', '胜负欲和追求特定设计', '实用性',
       '经典设计', '宠物用品', '满足个人兴趣', '满足日常需求', '为情怀买单', '节日礼物', '维持好心情',
       '搭配服饰', '怀旧情结', '购买礼物', '涨价前购买', '弥补过去遗憾', '纪念意义', '更换需求', '弥补遗憾',
       '分享与展示', '追求高品质', '追求实用性', '圆儿时梦想', '喜欢设计', '赠送他人, 自用', '满足功能性需求',
       '补充产品种类', '追求时尚搭配', '个人使用', '满足家庭成员需求', '购物分享', '为了扭蛋', '实用考虑',
       '意外收获', '追求热门商品', '经典永不过时', '追求梦想单品', '自我奖励/送礼', '赠送/代购', '欣赏美学设计',
       '收藏传承', '追求高雅气质', '拍照', '追求高质量', '追求特定设计', '个人装饰/摆件', '购买特定款

In [13]:
data_tagged.Purchase_Situation.unique()

array(['参观快闪店', '购物体验', nan, '单肩斜挎', '通勤', '新年礼物开箱', '打卡', '购物分享', '逛街',
       '周末活动', '日常使用', '逛街购物', '放松活动', '拍照打卡', '出街', '下午茶', '提前预定',
       '打卡拍照, 现场购买', '秋冬季节穿着', '收到礼物', '在线预订', '旅游购物', '海边度假', '春夏天使用',
       '日常出街', '个人试穿', '夏季穿着', '周末打卡', '日常分享', '生日购买', '出差和旅行', '年前',
       '初次购买体验', '日常穿搭', '快闪店购物', '晚上闲暇时间', '圣诞节/新年', '生日', '快闪店活动',
       '文创制作', '快闪店购买', '礼物赠送', '旅行通勤', '逛街拍照', '周末', '宠物穿搭', '日常出门',
       '第一时间购买', '下午茶后购物', '回市区购物', '买鞋时一并购买', '出门携带', '咖啡上海魔都探店', '春夏穿搭',
       '下班后购物', '畅想创作', '晚宴', '旅行打卡', '咖啡馆消费', '周末休闲', '店内购买', '网购',
       '过年最后一天', '日常购买', '现场购买', '逛街时顺便购买', '旅行', '前往快闪店购买', '线下店铺购买',
       '跨境购物', '逛马路、参观展览', '发售当天', '工作日参观', '日常搭配', '生日礼物', '过年', '新年穿搭',
       '代购/帮助他人购买', '生日月购买', '新年', '餐厅拍照', '日常出行', '购物', '日常带女儿逛街',
       '出国时候使用', '未提及', '快闪店排队购买', '逛店', '城市看展/周末活动', '周末出游', '上班', '约会',
       '限时展', '护理旧包', '不同场合灵活搭配', '过新年', '漫步城市街区', '工作日', '通勤、遛娃', '周末逛展',
       '儿童生日派对', '旅游', '取货时挑选购买', '和朋友见面', '街拍', '排队购买', '参加活动', '拍摄对比',

In [19]:
data_tagged.dtypes

内容                          object
raw_tagging                 object
parsed_json                 object
Age_Range                   object
Gender                      object
City_Tier                   object
Purchase_Purpose            object
Purchase_Situation          object
Product_Mention             object
Competitor                  object
Keywords_Design_Elements    object
Keywords_Design_Style       object
Keywords_Functionality      object
Keywords_Other              object
Emotion                     object
Purpose_Category            object
dtype: object

In [27]:
data_tagged.isnull().sum()

内容                             0
raw_tagging                    0
parsed_json                    0
Age_Range                   2479
Gender                      2125
City_Tier                   2081
Purchase_Purpose               0
Purchase_Situation             0
Product_Mention              572
Competitor                  2392
Keywords_Design_Elements    1231
Keywords_Design_Style       1427
Keywords_Functionality      1990
Keywords_Other                15
Emotion                      965
Purpose_Category               0
dtype: int64

In [ ]:
from typing import List
system_message = """
你是一名数据标注专家，现在需要对用户的购买需求进行归类，例如追求实用性，追求设计，情怀，社交需求等



注意：
不要出现除了类别名之外的任何内容，不要添加分类解释。


"""
chatbot = QWENChatBot(system_message=system_message)

'''
# 定义一个函数用于将帖子分类
def classify_post(content: str) -> dict:
    # 调用 QWEN 模型，得到响应
    try:
        if pd.isnull(content):  # Check if the content is null
            return "未明确提及" 
        return chatbot.ask(content)
    except Exception as e:
        response = f"Error: {str(e)}"
    
    return response
'''
def classify_post_thread_safe(content: str) -> str:
    try:
        if pd.isnull(content):  # Check if the content is null
            return "未明确提及" 
        return chatbot.ask(content)
    except Exception as e:
        return f"Error: {e}"
def classify_posts_concurrently(posts: List[str], max_workers: int = 5) -> List[str]:
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 将所有帖子分配给线程池并行处理
        futures = list(executor.map(classify_post_thread_safe, posts))
        results.extend(futures)
    return results

# 使用线程池处理 DataFrame 数据
data_tagged['Purpose_Category'] = classify_posts_concurrently(data_tagged['Purchase_Purpose'].tolist(), max_workers=16)
data_tagged['Purpose_Category'].value_counts()


Purpose_Category
未明确提及                                                                                       1455
追求设计                                                                                         443
情怀                                                                                           217
社交需求                                                                                         120
其他                                                                                           117
追求实用性                                                                                         93
设计                                                                                            85
实用性                                                                                           48
追求设计  \n追求实用性  \n\n（根据需求可二选一，如需单一类别，请确认主要倾向）                                                   2
追求设计\n\n社交需求                                                                                   2
情怀\n\n追求设计\n\

In [11]:
from typing import List
system_message = """
你是一名数据标注专家，现在需要对用户的购买场景进行归类，例如通勤，逛街，穿搭，旅游，节日等，无法精确归类的标注为其他

注意：
严格遵循数量限制，不要超过10个类别。
不要出现除了类别名之外的任何内容，不要添加任何分类解释。


"""
chatbot = QWENChatBot(system_message=system_message)

'''
# 定义一个函数用于将帖子分类
def classify_post(content: str) -> dict:
    # 调用 QWEN 模型，得到响应
    try:
        if pd.isnull(content):  # Check if the content is null
            return "未明确提及" 
        return chatbot.ask(content)
    except Exception as e:
        response = f"Error: {str(e)}"
    
    return response
'''
def classify_post_thread_safe(content: str) -> str:
    try:
        if pd.isnull(content):  # Check if the content is null
            return "未明确提及" 
        return chatbot.ask(content)
    except Exception as e:
        return f"Error: {e}"
def classify_posts_concurrently(posts: List[str], max_workers: int = 5) -> List[str]:
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 将所有帖子分配给线程池并行处理
        futures = list(executor.map(classify_post_thread_safe, posts))
        results.extend(futures)
    return results

# 使用线程池处理 DataFrame 数据
data_tagged['Situation_Category'] = classify_posts_concurrently(data_tagged['Purchase_Situation'].tolist(), max_workers=16)
data_tagged['Situation_Category'].value_counts()

Situation_Category
未明确提及                          1800
逛街                              269
其他                              239
节日                               85
穿搭                               81
旅游                               66
通勤                               54
通勤  \n逛街                          3
出街                                3
休闲                                2
逛街\n购物体验\n穿搭\n通勤\n节日\n其他          1
其他\n穿搭\n其他\n通勤\n节日\n其他            1
通勤  \n其他                          1
通勤  \n旅游                          1
其他\n日常使用\n逛街\n其他\n拍照打卡\n其他        1
通勤\n其他\n逛街\n节日\n其他                1
旅游\n其他\n旅游\n其他\n其他                1
逛街\n日常使用\n通勤\n其他                  1
其他\n日常使用\n通勤\n家居\n节日              1
节日\n其他                            1
旅游\n其他\n其他\n其他\n旅游                1
通勤\n其他\n节日\n其他\n逛街                1
逛街\n其他\n穿搭\n通勤                    1
逛街\n其他\n穿搭\n通勤\n节日                1
逛街\n其他\n日常使用\n拍照打卡\n出街\n下午茶       1
逛街\n其他\n日常使用\n其他\n穿搭              1
逛街\n日常使用\n逛街\n其他\n日常使用\n旅游        1
逛街\n其他\n逛

In [33]:
data_tagged[cols].to_excel('processed_data/tagged_processed.xlsx')

In [28]:
data_tagged.isnull().sum()/len(data_tagged)

内容                          0.000000
raw_tagging                 0.000000
parsed_json                 0.000000
Age_Range                   0.944021
Gender                      0.809216
City_Tier                   0.792460
Purchase_Purpose            0.000000
Purchase_Situation          0.000000
Product_Mention             0.217822
Competitor                  0.910891
Keywords_Design_Elements    0.468774
Keywords_Design_Style       0.543412
Keywords_Functionality      0.757807
Keywords_Other              0.005712
Emotion                     0.367479
Purpose_Category            0.000000
dtype: float64

In [32]:
cols = data_tagged.columns[:-1]

In [12]:
for col in data_tagged.iloc[:,3:].columns:
    print(data_tagged[col].value_counts()[:5])

Age_Range
25-34岁    75
45岁以上     35
35-44岁    23
18-24岁    12
18岁以下      2
Name: count, dtype: int64
Gender
女性    471
男性     30
Name: count, dtype: int64
City_Tier
一线       497
二线        40
三线及以下      8
Name: count, dtype: int64
Purchase_Purpose
追求独特设计    332
收藏         55
追求时尚潮流     46
个人喜好       38
追求时尚       31
Name: count, dtype: int64
Purchase_Situation
日常出街    121
日常使用     86
逛街       24
日常穿搭     22
通勤       17
Name: count, dtype: int64
Product_Mention
[白三彩]         122
[包包]           21
[太阳花]          16
[carryall]     14
[speedy]       14
Name: count, dtype: int64
Competitor
[爱马仕]       11
[chanel]     7
[Chanel]     6
[miumiu]     4
[香奈儿]        4
Name: count, dtype: int64
Keywords_Design_Elements
[白三彩]     157
[太阳花]      37
[黑三彩]      35
[新配色]      16
[黑白三彩]     12
Name: count, dtype: int64
Keywords_Design_Style
[可爱]    164
[好看]     63
[时尚]     45
[经典]     30
[百搭]     22
Name: count, dtype: int64
Keywords_Functionality
[实用]      18
[百搭]      13
[小而不废]     7
[斜挎]       5
[好闻] 

In [13]:
from collections import Counter
def count_word_freq(column):
    # Flatten all lists from the column into a single list, ignoring NaN values
    all_keywords = [word for keywords in column.dropna() for word in keywords]
    
    # Count the frequency of each word using Counter
    return Counter(all_keywords)
# Function to get top N frequent keywords
def get_top_n(counter_obj, n=5):
    return counter_obj.most_common(n)


# Count word frequencies for each subcategory
freq_product = count_word_freq(data_tagged['Product_Mention'])
freq_competitor = count_word_freq(data_tagged['Competitor'])
freq_design_elements = count_word_freq(data_tagged['Keywords_Design_Elements'])
freq_design_style = count_word_freq(data_tagged['Keywords_Design_Style'])
freq_functionality = count_word_freq(data_tagged['Keywords_Functionality'])
freq_other = count_word_freq(data_tagged['Keywords_Other'])

# Display top 5 keywords for each subcategory
print("Design Elements Word Frequency (Top 5):")
print(get_top_n(freq_product, 5))
print("Design Elements Word Frequency (Top 5):")
print(get_top_n(freq_competitor, 5))
print("Design Elements Word Frequency (Top 5):")
print(get_top_n(freq_design_elements, 5))

print("\nDesign Style Word Frequency (Top 5):")
print(get_top_n(freq_design_style, 5))

print("\nFunctionality Word Frequency (Top 5):")
print(get_top_n(freq_functionality, 10))

print("\nOther Word Frequency (Top 5):")
print(get_top_n(freq_other, 5))

Design Elements Word Frequency (Top 5):
[('白三彩', 359), ('黑三彩', 95), ('speedy', 72), ('carryall', 61), ('包包', 44)]
Design Elements Word Frequency (Top 5):
[('Chanel', 18), ('爱马仕', 17), ('miumiu', 13), ('chanel', 12), ('香奈儿', 10)]
Design Elements Word Frequency (Top 5):
[('白三彩', 403), ('黑三彩', 135), ('太阳花', 78), ('三彩', 65), ('白色', 62)]

Design Style Word Frequency (Top 5):
[('可爱', 268), ('时尚', 110), ('好看', 87), ('经典', 74), ('复古', 54)]

Functionality Word Frequency (Top 5):
[('斜挎', 46), ('实用', 42), ('容量大', 23), ('能装', 22), ('手拎', 20), ('百搭', 20), ('单肩', 19), ('大容量', 15), ('手提', 11), ('小而不废', 7)]

Other Word Frequency (Top 5):
[('村上隆', 1259), ('联名系列', 819), ('lv村上隆', 363), ('LV', 155), ('联名', 100)]


In [ ]:
from typing import List
9.根据帖子内容，结合以上信息将用户分为不超过6种类型，并总结每种类型人群的特点：
用户分类（Segmentation）：例如：“潮流先锋”、“奢侈品爱好者”；类型名称应通俗易懂，清晰明了，每类类型不能高度重合，各个分类须体现差异化特点。
类型描述（Segmentation_Description）：描述中体现人群的核心兴趣或行为特点（如对品牌热爱、艺术感知力、消费习惯等），例如：“对潮流有高度敏感性，热衷于分享热点信息”。
system_message = """
你是一名数据标注专家，现在需要对用户发布的关于 LVMH 和村上隆联名系列的小红书帖子进行分类。请根据以下规则，将用户分为不超过6种类型，并总结每种类型人群的特点。

输出要求：

每种用户类型的名称，例如：“潮流先锋”、“奢侈品爱好者”；
每种类型的简短描述，突出人群特点，例如：“对潮流有高度敏感性，热衷于分享热点信息”。

规则：

类型名称应通俗易懂，清晰明了。
描述中体现人群的核心兴趣或行为特点（如对品牌热爱、艺术感知力、消费习惯等）。
注意：

每类类型不能高度重合，各个分类须体现差异化特点。
务必严格遵守类型数量限制，不得超过6种。

请严格按照以下json形式输出结果：
{
    "segmentation": "类型名称",
    "description": "该类型人群的简短特点描述"
  }

"""
chatbot = QWENChatBot(system_message=system_message)

'''
# 定义一个函数用于将帖子分类
def classify_post(content: str) -> dict:
    # 调用 QWEN 模型，得到响应
    try:
        response = chatbot.ask(content)
    except Exception as e:
        response = f"Error: {str(e)}"
    
    return response
'''
def classify_post_thread_safe(content: str) -> int:
    try:
        return chatbot.ask(content)
    except Exception as e:
        return f"Error: {e}"
def classify_posts_concurrently(posts: List[str], max_workers: int = 5) -> List[int]:
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 将所有帖子分配给线程池并行处理
        futures = list(executor.map(classify_post_thread_safe, posts))
        results.extend(futures)
    return results

# 使用线程池处理 DataFrame 数据
data_tagged['classification'] = classify_posts_concurrently(data_tagged['内容'].tolist(), max_workers=16)
data_tagged
